
# Preparing data for ingestion into a database for Retrieval Augmented Generation.
AGROVOC and Agriprod are both structed datasets (excel sheets) that must be turned into prose like structure.
Here we choose to serialize them row wise and change the column names into more meaningful headings for the best performance
The column conceptLabel_en/Name_EN is the crop description in English. This is reused in the serialization of every single column to further emphasize the relationship

In [7]:
# first agrovoc file including crops and plant products

import pandas as pd
import json


In [ ]:
# function to convert the excel entries into text by row while including the column names into the text

def serialize_df(df: pd.DataFrame) -> list[str]:
    """
    Convert each row of a dataframe into a string of the form:
    "<first_column>'s <col> is <value>, ..."
    e.g <Brussels sprouts>'s <german translation> is <Rosenkohl>
    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to process. The first column is treated as the "entity".

    Returns
    -------
    list[str]
        A list of formatted strings for each row.
    """
    if df.empty or len(df.columns) < 2:
        return []

    first_col = df.columns[0]

    rows = []
    for _, row in df.iterrows():
        parts = []
        for col in df.columns[1:]:
            val = row[col]
            if pd.notna(val) and str(val).strip() != "":
                parts.append(f"{row[first_col]}'s {col} is {val}")
        rows.append(", ".join(parts))

    return rows

In [9]:

agrovoc_df1 = pd.read_csv('/home/eouser/tabVol/automate_eurocrops/data/raw/Query_multilingual_includes_useof.csv')

In [10]:
# remove the first column that only contain url links
agrovoc_df1 = agrovoc_df1.drop('concept', axis=1)

# since the original column names are not very informative, we convert them to something that serializes better
agrovoc_df1 = agrovoc_df1.rename(columns={'conceptLabel_en':'english label', 'conceptLabel_de':'german translation', 'conceptLabel_fr':'french translation','conceptDef_en': 'english definition', 'conceptDef_de':'german definition', 'conceptDef_fr':'french definition', 'altLabel_en': 'synonym',
       'altLabel_de': 'german synonym', 'altLabel_fr':'french synonym', 'includesLabel_en':'includes', 'includesLabel_de':'includes in german',
       'includesLabel_fr':'includes in french', 'isUseOfLabel_en':'use of', 'isUseOfLabel_de':'use of in german',
       'isUseOfLabel_fr':'use of in french', 'producesLabel_en':'produce', 'producesLabel_de':'produce in german',
       'producesLabel_fr':'produce in french', 'isProducedByLabel_en':'produced by', 'isProducedByLabel_de':'produced by in german',
       'isProducedByLabel_fr':'produced by in french'  })
# serialize into rows of strings
df1_rows = serialize_df(agrovoc_df1 )       

In [11]:
# continue with the next agrovoc file
# this includes ornamental plants and forest products 
agrovoc_df2 = pd.read_csv('/home/eouser/tabVol/automate_eurocrops/data/raw/Query_ornamentals_trees.csv')
agrovoc_df2 = agrovoc_df2.drop('concept', axis=1)
agrovoc_df2 = agrovoc_df2.rename(columns={'conceptLabel_en':'english label', 'conceptLabel_de':'german translation', 'conceptLabel_fr':'french translation','conceptDef_en': 'english definition', 'conceptDef_de':'german definition', 'conceptDef_fr':'french definition', 'altLabel_en': 'synonym',
       'altLabel_de': 'german synonym', 'altLabel_fr':'french synonym', 'includesLabel_en':'includes', 'includesLabel_de':'includes in german',
       'includesLabel_fr':'includes in french', 'isUseOfLabel_en':'use of', 'isUseOfLabel_de':'use of in german',
       'isUseOfLabel_fr':'use of in french', 'producesLabel_en':'produce', 'producesLabel_de':'produce in german',
       'producesLabel_fr':'produce in french', 'isProducedByLabel_en':'produced by', 'isProducedByLabel_de':'produced by in german',
       'isProducedByLabel_fr':'produced by in french'  })
df2_rows =    serialize_df(agrovoc_df2)      

In [12]:
# now load agriprod dataset
agriprod_df = pd.read_excel('/home/eouser/tabVol/automate_eurocrops/data/raw/AGRIPROD_Codes_Names_Notes_crops_only.xlsx')

# remove the special symbols in theagriprod dataset
agriprod_df = agriprod_df.applymap(lambda x: x.replace("\n•", "") if isinstance(x, str) else x)
agriprod_df = agriprod_df.applymap(lambda x: x.replace("\n o", "") if isinstance(x, str) else x)

# drop the irrelevant columns 
agriprod_df = agriprod_df.drop(['Code','IFS_T_INPUT', 'IFS_T_DERIVED'] , axis=1)

# rename for better flow
agriprod_df = agriprod_df.rename(columns={'Name_EN':'english name', 'Name_FR':'french name', 'Name_DE': 'german name' })

# serialize
agriprod_strings =serialize_df(agriprod_df)

/tmp/ipykernel_877972/2532898190.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  agriprod_df = agriprod_df.applymap(lambda x: x.replace("\n•", "") if isinstance(x, str) else x)
/tmp/ipykernel_877972/2532898190.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  agriprod_df = agriprod_df.applymap(lambda x: x.replace("\n o", "") if isinstance(x, str) else x)


In [13]:
# combine all 3 datasets
combined_strings = df1_rows+df2_rows+agriprod_strings

In [14]:
with open('/home/eouser/tabVol/automate_eurocrops/data/processed/agrovoc_agriprod_database.txt', 'w', encoding='utf-8') as f:
    for row in combined_strings:

        f.write(row + '\n')